<a href="https://colab.research.google.com/github/MiniRedTrout/Segmentation/blob/main/notebook4e08fe2ac5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт библиотек

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import os
import scipy.io
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import shutil

Путь датасета с kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("miniredtrout/abdominalaorta")

Using Colab cache for faster access to the 'abdominalaorta' dataset.


In [ ]:
path

'/kaggle/input/abdominalaorta'

In [ ]:
!pip install hydra-core omegaconf
import hydra
from omegaconf import DictConfig, OmegaConf
import os

In [ ]:
!pip install clearml

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=XGMS61QRIXH7G4URU0X7VGTDAIQUVE
%env CLEARML_API_SECRET_KEY=3mpOB_Br37BLBntBoUvtxmfbBfmkbkdfRUYGje8GfL1gwe_ufJd1w4UzuQWOPaAtTWE

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=XGMS61QRIXH7G4URU0X7VGTDAIQUVE
env: CLEARML_API_SECRET_KEY=3mpOB_Br37BLBntBoUvtxmfbBfmkbkdfRUYGje8GfL1gwe_ufJd1w4UzuQWOPaAtTWE


In [ ]:
from clearml import Task, Logger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import hydra
from omegaconf import DictConfig, OmegaConf

config_dir = '/content/drive/MyDrive/segmentation_project/config'

# Очистим и пересоздадим папку
import shutil
if os.path.exists(config_dir):
    shutil.rmtree(config_dir)

os.makedirs(config_dir, exist_ok=True)

simple_config = """
training:
  batch_size: 16
  epochs: 40
  learning_rate: 0.0002
  optimizer: "AdamW"
  weight_decay: 0.01

data:
  original_images_dir: "/kaggle/input/abdominalaorta/images"
  original_labels_dir: "/kaggle/input/abdominalaorta/labels"
  output_base_dir: "/content/base_patients"
  test_size: 0.2
  random_state: 42
  image_size: 256
  num_workers: 2

model:
  in_channel: 1
  out_channel: 2

transforms:
  image_size: 256
"""

with open(f'{config_dir}/config.yaml', 'w') as f:
    f.write(simple_config)

print("✅ Простой конфиг создан!")

def load_hydra_config():
    config_path = "/content/drive/MyDrive/segmentation_project/config"
    config_name = "config"

    with hydra.initialize_config_dir(config_dir=config_path, version_base="1.2"):
        cfg = hydra.compose(config_name=config_name)

    # ОТКЛЮЧАЕМ STRICT MODE ПРОГРАММНО
    OmegaConf.set_struct(cfg, False)

    return cfg

Mounted at /content/drive
✅ Простой конфиг создан!


Разделение на train test выборку по пациентам

In [ ]:
def create_train_test_split_by_patient():
    cfg = load_hydra_config()
    patient_folders = []
    for folder in os.listdir(cfg.data.original_images_dir):
        folder_path = os.path.join(cfg.data.original_images_dir, folder)
        if (os.path.isdir(folder_path) and
            os.path.exists(os.path.join(cfg.data.original_labels_dir, folder))):
            patient_folders.append(folder)

    train_patients, test_patients = train_test_split(
        patient_folders,
        test_size=cfg.data.test_size,
        random_state=cfg.data.random_state
    )

    train_images_dir = os.path.join(cfg.data.output_base_dir, 'train', 'images')
    train_labels_dir = os.path.join(cfg.data.output_base_dir, 'train', 'labels')
    test_images_dir = os.path.join(cfg.data.output_base_dir, 'test', 'images')
    test_labels_dir = os.path.join(cfg.data.output_base_dir, 'test', 'labels')

    for dir_path in [train_images_dir, train_labels_dir, test_images_dir, test_labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    def copy_patient_files(patients, src_images_dir, src_labels_dir, dst_images_dir, dst_labels_dir):
        total_files = 0
        for patient in patients:
            patient_images_dir = os.path.join(src_images_dir, patient)
            patient_labels_dir = os.path.join(src_labels_dir, patient)

            for file in os.listdir(patient_images_dir):
                if file.endswith('.png'):
                    src_image_path = os.path.join(patient_images_dir, file)
                    dst_image_path = os.path.join(dst_images_dir, f"{patient}_{file}")
                    shutil.copy2(src_image_path, dst_image_path)

                    src_label_path = os.path.join(patient_labels_dir, file)
                    dst_label_path = os.path.join(dst_labels_dir, f"{patient}_{file}")
                    shutil.copy2(src_label_path, dst_label_path)

                    total_files += 1

        return total_files

    train_total = copy_patient_files(train_patients, cfg.data.original_images_dir, cfg.data.original_labels_dir,
                                   train_images_dir, train_labels_dir)

    test_total = copy_patient_files(test_patients, cfg.data.original_images_dir, cfg.data.original_labels_dir,
                                  test_images_dir, test_labels_dir)

    return {
        'train_images': train_images_dir,
        'train_labels': train_labels_dir,
        'test_images': test_images_dir,
        'test_labels': test_labels_dir,
        'train_patients': train_patients,
        'test_patients': test_patients
    }

Датасет

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
        self.label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.png')])
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        label_path = os.path.join(self.labels_dir, self.label_files[idx])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image']
            label = transformed['mask']
        label = (label > 0).long()

        return image, label

Модель Unet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def contracting_block(self, in_channels, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=out_channels, out_channels=out_channels, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(out_channels),
        )
        return block

    def expansive_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.ConvTranspose2d(in_channels=mid_channel, out_channels=out_channels, kernel_size=2, stride=2)
        )
        return block

    def final_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=out_channels, padding=1),
        )
        return block

    def __init__(self, in_channel, out_channel):
        super(UNet, self).__init__()
        self.conv_encode1 = self.contracting_block(in_channels=in_channel, out_channels=64)
        self.conv_maxpool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode2 = self.contracting_block(64, 128)
        self.conv_maxpool2 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode3 = self.contracting_block(128, 256)
        self.conv_maxpool3 = torch.nn.MaxPool2d(kernel_size=2)

        self.bottleneck = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=3, in_channels=256, out_channels=512, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(512),
            torch.nn.Conv2d(kernel_size=3, in_channels=512, out_channels=512, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(512),
            torch.nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2)
        )

        self.conv_decode3 = self.expansive_block(512, 256, 128)
        self.conv_decode2 = self.expansive_block(256, 128, 64)
        self.final_layer = self.final_block(128, 64, out_channel)

    def crop_and_concat(self, upsampled, bypass):
        _, _, h_up, w_up = upsampled.size()
        _, _, h_by, w_by = bypass.size()

        crop_h = (h_by - h_up) // 2
        crop_w = (w_by - w_up) // 2

        bypass_cropped = bypass[:, :,
                              crop_h:crop_h + h_up,
                              crop_w:crop_w + w_up]

        return torch.cat((upsampled, bypass_cropped), 1)

    def forward(self, x):
        encode_block1 = self.conv_encode1(x)
        encode_pool1 = self.conv_maxpool1(encode_block1)

        encode_block2 = self.conv_encode2(encode_pool1)
        encode_pool2 = self.conv_maxpool2(encode_block2)

        encode_block3 = self.conv_encode3(encode_pool2)
        encode_pool3 = self.conv_maxpool3(encode_block3)

        bottleneck1 = self.bottleneck(encode_pool3)

        decode_block3 = self.crop_and_concat(bottleneck1, encode_block3)
        cat_layer2 = self.conv_decode3(decode_block3)

        decode_block2 = self.crop_and_concat(cat_layer2, encode_block2)
        cat_layer1 = self.conv_decode2(decode_block2)

        decode_block1 = self.crop_and_concat(cat_layer1, encode_block1)
        final_layer = self.final_layer(decode_block1)

        return final_layer

Аугментация и предподготовка

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

def transformers():
    cfg = load_hydra_config()
    train_transform = A.Compose([
        A.Resize(height=cfg.transforms.image_size, width=cfg.transforms.image_size),

        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.1,
            rotate_limit=10,
            p=0.5
        ),
        A.RandomBrightnessContrast(
            brightness_limit=0.1,
            contrast_limit=0.1,
            p=0.3
        ),
        A.GaussNoise(var_limit=(10.0, 30.0), mean=0, p=0.2),

        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=cfg.transforms.image_size, width=cfg.transforms.image_size),
        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2(),
    ])

    return train_transform, val_transform

In [ ]:
def dice_score_ce(predictions, targets, smooth=1e-6):
    probs = torch.softmax(predictions, dim=1)

    pred_probs = probs[:, 1]

    pred_binary = torch.argmax(probs, dim=1)

    pred_flat = pred_binary.flatten(1)
    target_flat = targets.flatten(1)
    intersection = (pred_flat * target_flat).sum(1)
    union = pred_flat.sum(1) + target_flat.sum(1)

    dice = (2. * intersection + smooth) / (union + smooth)

    return dice.mean()

In [ ]:
!pip install torchmetrics
from torchmetrics import JaccardIndex

from torchmetrics.segmentation import DiceScore

from torchmetrics.classification import BinaryPrecision, BinaryRecall, BinaryF1Score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 24.2 MB/s eta 0:00:00


In [ ]:
iou_metric = JaccardIndex(task='binary')  # или task='multiclass', num_classes=2
dice_metric = DiceScore(num_classes=2, average='micro', input_format='index')
precision_metric = BinaryPrecision()
recall_metric = BinaryRecall()
f1_metric = BinaryF1Score()


/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: DiceScore metric currently defaults to `average=micro`, but will change to`average=macro` in the v1.9 release. If you've explicitly set this parameter, you can ignore this warning.
  warnings.warn(*args, **kwargs)


In [ ]:
def train_model():
    cfg = load_hydra_config()
    task = Task.init(
        project_name="MedicalSegmentation",
        task_name=f"UNet_{cfg.training.epochs}epochs",
        auto_connect_frameworks={'pytorch': True, 'hydra': True}
    )
    cfg_dict = OmegaConf.to_container(cfg, resolve=True)
    task.connect_configuration(cfg_dict)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    paths = create_train_test_split_by_patient()
    train_transform, val_transform = transformers()
    train_dataset = SegmentationDataset(
        images_dir=paths['train_images'],
        labels_dir=paths['train_labels'],
        transform=train_transform
    )

    test_dataset = SegmentationDataset(
        images_dir=paths['test_images'],
        labels_dir=paths['test_labels'],
        transform=val_transform
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.training.batch_size,
        shuffle=True,
        num_workers=cfg.data.num_workers
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.training.batch_size,
        shuffle=False,
        num_workers=cfg.data.num_workers
    )

    model = UNet(cfg.model.in_channel, cfg.model.out_channel).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=cfg.training.learning_rate,
        weight_decay=cfg.training.weight_decay
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                          mode='min')
    train_losses = []
    val_dice_scores = []
    for epoch in range(cfg.training.epochs):
        model.train()
        epoch_train_loss = 0
        train_pbar = tqdm(train_loader,desc=f'Epoch {epoch}/{cfg.training.epochs} [Train]')
        for idx, (images, labels) in enumerate(train_pbar):
            images, labels = images.to(device), labels.to(device)
            labels = labels.long()
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_train_loss += loss.item()

            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{epoch_train_loss/(idx+1):.4f}'
            })
        train_pbar.close()
        avg_train_loss = epoch_train_loss / len(train_loader)
        model.eval()
        epoch_val_dice = 0
        val_batches = 0
        val_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{cfg.training.epochs} [Val]')
        with torch.no_grad():
            for idx, (images, masks) in enumerate(val_pbar):
                images, masks = images.to(device), masks.to(device)
                masks = masks.long()

                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)
                preds = torch.argmax(probs, dim=1)
                masks = masks.long()
                masks = masks.unsqueeze(1)
                preds = preds.squeeze(1)
                dice_metric.update(outputs, masks)

                val_batches += 1
                val_pbar.set_postfix({
                    'Dice': f'{dice_metric.item():.4f}',
                    'Avg Dice': f'{epoch_val_dice/(idx+1):.4f}'
                })

        val_pbar.close()

        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_val_dice = dice_metric.compute()
        logger = Logger.current_logger()
        logger.report_scalar("Loss", "Train", avg_train_loss, epoch)
        logger.report_scalar("Metrics", "Dice", avg_val_dice, epoch)
        train_losses.append(avg_train_loss)
        val_dice_scores.append(avg_val_dice)
        scheduler.step(avg_train_loss)
        print(f'Epoch {epoch:03d}:')
        print(f'  Loss: {avg_train_loss:.4f}')
        print(f'  Dice: {avg_val_dice:.4f}')
        print(f'  LR:   {optimizer.param_groups[0]["lr"]:.2e}')
        print('-' * 40)



In [ ]:
if __name__ == "__main__":
    results = train_model()

/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning:

ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.

/tmp/ipython-input-2562268279.py:24: UserWarning:

Argument(s) 'var_limit, mean' are not valid for transform GaussNoise

Epoch 0/40 [Train]:  92%|█████████▏| 229/250 [02:56<00:15,  1.33it/s, Loss=0.0937, Avg Loss=0.3684]